In [2]:
import json
import os
import google.generativeai as genai
from google.generativeai import caching
import datetime
import time
from dotenv import load_dotenv

load_dotenv()
QUERY_UUID="92c5771b-8f73-4790-8fb2-3f35f492d0fe"

PROMPT="""You are a research assistant who classifies studies. You are tasked with filling out a PRISMA flow diagram based on uploaded PDFs. For each PDF, you are to tell me at what level was the paper is filtered out, or if it should be included. Be very critical.

###PRISMA Explaination###

This document is a PRISMA 2020 flow diagram.  PRISMA stands for Preferred Reporting Items for Systematic Reviews and Meta-Analyses.  This specific diagram is a template for researchers to visually represent the flow of information through the different phases of a systematic review.  It helps ensure transparency and completeness in reporting how a review was conducted.

Here's a breakdown of what the diagram represents:

1. **Identification:** This stage shows how many records (e.g., research articles) were identified through different sources like databases, registers, and other sources (e.g., hand-searching reference lists, contacting experts). The diagram encourages reporting the number from each source if feasible.

2. **Screening:** After identification, records are screened based on pre-defined inclusion and exclusion criteria.  The diagram shows how many records were removed at this stage and the reasons for exclusion (e.g., wrong study type, wrong population).  It also prompts authors to specify if automation tools were used and how many records were excluded by humans versus automation.

3. **Eligibility:**  The remaining records after screening are assessed for eligibility. This often involves reading the full text of the articles to confirm they meet all inclusion criteria.  Again, the diagram tracks how many records were excluded at this stage and the reasons why.

4. **Included:** Finally, the diagram shows the number of studies that were ultimately included in the systematic review.  These are the studies that were considered eligible and provided data relevant to the research question. For a study to be included, the intervention must be included.

Essentially, the PRISMA flow diagram provides a clear, visual representation of the search and selection process in a systematic review.  By following this diagram, researchers can document their methods thoroughly and ensure readers understand how they arrived at the final set of included studies.  This helps minimize bias and improve the reliability of the review's findings.  The diagram also explicitly references the original source and its licensing information.

### Inclusion Criteria###

Here is the inclusion exclusion criteria. 

Here is the inclusion criteria for a study.


Inclusion and Exclusion Criteria
Inclusion Criteria

Population:

    Age: 65 years or older
    Diagnosis of depression or depressive symptons: Confirmed by a healthcare professional using a standardized diagnostic tool (e.g., DSM-5, ICD-10)

Intervention/Exposure:

    Regular aerobic exercise:
        Frequency: At least 3 times per week
        Intensity: Moderate to vigorous intensity
        Duration: At least 30 minutes per session
        Type: Structured exercise programs (e.g., walking, running, cycling)

Comparator:

    Usual care: Standard treatment for depression (e.g., medication, psychotherapy)
    No exercise intervention: No specific exercise program

Outcome:

    Reduction in depression symptoms: Measured by a standardized depression scale (e.g., Beck Depression Inventory, Hamilton Depression Rating Scale) as a primary or secondary outcome.

Study Design:

    Randomized controlled trials (RCTs) or Observational Studies
Exclusion Criteria

    Severe medical conditions that preclude exercise (e.g., unstable heart disease, severe respiratory conditions)
    Cognitive impairment that interferes with informed consent or participation in the study
    Current use of psychotropic medications that could confound the effects of exercise
    History of substance abuse or dependence
    Pregnancy or breastfeeding
    Participation in other ongoing exercise or treatment programs for depression
    A systematic reveiw or meta-analysis
"""


genai.configure(api_key=os.environ['GEMINI_API_KEY'])

base = f"pdfs/{QUERY_UUID}/"
paths = os.listdir(base)
files = []

for path in paths:
    files.append(genai.upload_file(path=base+path))

num_papers = len(paths)

# Create a cache with a 5 minute TTL
cache = caching.CachedContent.create(
    model='models/gemini-1.5-flash-001',
    display_name='test', # used to identify the cache
    system_instruction=(PROMPT),
    contents=files,
    ttl=datetime.timedelta(minutes=5),
)



model = genai.GenerativeModel.from_cached_content(cached_content=cache)

response = model.generate_content([(
    f"There are {num_papers} articles in this research. Please return a json array with the title of each of the articles."
)])

paper_titles = json.loads(response.text)

response = model.generate_content([(
    "For each of these articles, please List if it meets each of the PICO/PECO criteria, and then classify them as passing or failing. It must pass every point to be included. Allow for Interventions and Exposures. A study should be included if the PICO/PECO question can be answered by the data (e.g., a study of adults, which reports out the age group of 65+ would still be included). All for the studies which measure association as well a causation. "
)])

print(response.usage_metadata)
print(response.text)


ServiceUnavailable: 503 The service is currently unavailable.